In [1]:
import os
import re
import pandas as pd
import plotly.graph_objects as go
from sqlalchemy import create_engine
from dotenv import load_dotenv

from src.utils.file_utils import convert_time_to_seconds


# -----------------------------------------------------------------------------
# Connexion DB (même logique que app.py)
# -----------------------------------------------------------------------------
def get_engine_from_config():
    try:
        import streamlit as st
        db_url = st.secrets["DB_URL"]
    except Exception:
        load_dotenv()
        db_url = os.getenv("DB_URL")

    if not db_url:
        raise ValueError("DB_URL introuvable dans st.secrets ou .env")

    return create_engine(db_url)


# -----------------------------------------------------------------------------
# Helpers
# -----------------------------------------------------------------------------
def is_birth_date_incomplete(birth_date_raw: str) -> bool:
    """Date complète attendue: JJ/MM/AAAA. Sinon => incomplète."""
    if birth_date_raw is None or pd.isna(birth_date_raw):
        return True
    return re.fullmatch(r"\d{2}/\d{2}/\d{4}", str(birth_date_raw).strip()) is None


def resolve_athlete_candidates(engine, athlete_identifier: str, limit: int = 10) -> pd.DataFrame:
    """
    Résout un identifiant d'athlète donné par seq OU par nom (partiel).
    Ex: 'DIRAT', 'VAUDOIS', ou un seq exact.
    """
    query = """
        SELECT
            seq,
            name,
            birth_date_raw,
            birth_year
        FROM athletes
        WHERE seq = %(identifier)s
           OR name ILIKE %(name_like)s
        ORDER BY
            CASE
                WHEN seq = %(identifier)s THEN 0
                WHEN LOWER(name) = LOWER(%(identifier)s) THEN 1
                ELSE 2
            END,
            name
        LIMIT %(limit)s
    """

    return pd.read_sql_query(
        query,
        engine,
        params={
            "identifier": athlete_identifier,
            "name_like": f"%{athlete_identifier}%",
            "limit": limit,
        },
    )


def fetch_athlete_800m_by_seq(engine, seq: str) -> pd.DataFrame:
    query = """
        SELECT
            r.seq,
            a.name,
            a.birth_date_raw,
            a.birth_year,
            r.date,
            r.epreuve,
            r.perf,
            r.ville,
            r.tour
        FROM results r
        LEFT JOIN athletes a ON a.seq = r.seq
        WHERE r.seq = %(seq)s
          AND r.epreuve ILIKE '800m%%'
    """
    return pd.read_sql_query(query, engine, params={"seq": seq})


def fetch_best_match_athlete_800m(engine, athlete_identifier: str) -> pd.DataFrame:
    """
    1) Tente de résoudre via athletes(seq/nom).
    2) Prend le candidat qui possède le plus de lignes 800m.
    3) Fallback: considère l'identifiant comme seq brut.
    """
    candidates = resolve_athlete_candidates(engine, athlete_identifier)

    best_df = pd.DataFrame()
    best_count = -1

    if not candidates.empty:
        for _, row in candidates.iterrows():
            seq = row["seq"]
            candidate_df = fetch_athlete_800m_by_seq(engine, seq)
            count = len(candidate_df)
            if count > best_count:
                best_count = count
                best_df = candidate_df

    if best_df.empty:
        best_df = fetch_athlete_800m_by_seq(engine, athlete_identifier)

    return best_df


def prepare_for_comparison(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df

    out = df.copy()
    out["date"] = pd.to_datetime(out["date"], errors="coerce")
    out["time_sec"] = out["perf"].apply(convert_time_to_seconds)
    out = out.dropna(subset=["date", "time_sec"]).sort_values("date")

    out["year"] = out["date"].dt.year
    out["birth_year"] = pd.to_numeric(out["birth_year"], errors="coerce")
    out["age"] = out["year"] - out["birth_year"]

    # Règle demandée:
    # si date de naissance incomplète => ne garder que la meilleure perf par âge
    birth_raw = out["birth_date_raw"].dropna().iloc[0] if out["birth_date_raw"].notna().any() else None
    if is_birth_date_incomplete(birth_raw):
        out = out.dropna(subset=["age"]).copy()
        if not out.empty:
            idx_best = out.groupby("age")["time_sec"].idxmin()
            out = out.loc[idx_best].sort_values("age")

    return out


def add_trace(fig, dfa: pd.DataFrame, athlete_label: str, x_col: str, visible: bool):
    fig.add_trace(
        go.Scatter(
            x=dfa[x_col],
            y=dfa["time_sec"],
            mode="lines+markers",
            name=athlete_label,
            visible=visible,
            text=dfa["perf"],
            customdata=dfa[["date", "age", "year"]],
            hovertemplate=(
                "Athlète: " + athlete_label + "<br>"
                + "Perf: %{text}<br>"
                + "Date: %{customdata[0]|%Y-%m-%d}<br>"
                + "Âge: %{customdata[1]:.0f}<br>"
                + "Année: %{customdata[2]:.0f}<extra></extra>"
            ),
        )
    )


def plot_compare_2_athletes_800m(athlete_1: str, athlete_2: str):
    """
    athlete_1 / athlete_2 acceptent:
    - un seq exact
    - ou un nom/fragment de nom (ex: 'DIRAT', 'VAUDOIS')
    """
    engine = get_engine_from_config()

    a1_raw = fetch_best_match_athlete_800m(engine, athlete_1)
    a2_raw = fetch_best_match_athlete_800m(engine, athlete_2)

    a1 = prepare_for_comparison(a1_raw)
    a2 = prepare_for_comparison(a2_raw)

    if a1.empty or a2.empty:
        missing = []
        if a1.empty:
            missing.append(athlete_1)
        if a2.empty:
            missing.append(athlete_2)
        raise ValueError(
            "Aucune donnée 800m disponible pour: " + ", ".join(missing)
            + "\nVérifie que ces athlètes existent dans la table athletes/results."
        )

    name_1 = a1["name"].dropna().iloc[0] if a1["name"].notna().any() else athlete_1
    name_2 = a2["name"].dropna().iloc[0] if a2["name"].notna().any() else athlete_2

    # Pour le mode X=âge, on retire les lignes sans âge
    a1_age = a1.dropna(subset=["age"]).sort_values("age")
    a2_age = a2.dropna(subset=["age"]).sort_values("age")

    # Pour le mode X=année
    a1_year = a1.dropna(subset=["year"]).sort_values("year")
    a2_year = a2.dropna(subset=["year"]).sort_values("year")

    fig = go.Figure()

    # Traces visibles par défaut: mode Âge
    add_trace(fig, a1_age, name_1, x_col="age", visible=True)
    add_trace(fig, a2_age, name_2, x_col="age", visible=True)

    # Traces cachées: mode Année
    add_trace(fig, a1_year, name_1, x_col="year", visible=False)
    add_trace(fig, a2_year, name_2, x_col="year", visible=False)

    fig.update_layout(
        title="Comparaison de l'évolution des performances (800m)",
        xaxis_title="Âge",
        yaxis_title="Temps (secondes)",
        template="plotly_white",
        hovermode="x unified",
        updatemenus=[
            {
                "type": "dropdown",
                "direction": "down",
                "x": 1.02,
                "y": 1.12,
                "showactive": True,
                "buttons": [
                    {
                        "label": "X = Âge",
                        "method": "update",
                        "args": [
                            {"visible": [True, True, False, False]},
                            {"xaxis": {"title": "Âge"}},
                        ],
                    },
                    {
                        "label": "X = Année",
                        "method": "update",
                        "args": [
                            {"visible": [False, False, True, True]},
                            {"xaxis": {"title": "Année"}},
                        ],
                    },
                ],
            }
        ],
    )

    # Axe Y inversé (valeurs les plus grandes en haut)
    fig.update_yaxes(autorange=True)

    return fig


# -----------------------------------------------------------------------------
# Exemple d'utilisation
# -----------------------------------------------------------------------------
ATHLETE_1 = "VAUDOIS"   # seq ou nom
ATHLETE_2 = "DIRAT"     # seq ou nom

fig = plot_compare_2_athletes_800m(ATHLETE_1, ATHLETE_2)
fig.show()

In [ ]:
# Cellule 2 : même graphique + sélection dynamique des courses 100m -> Marathon
import ipywidgets as widgets
from IPython.display import display, clear_output


# Épreuves autorisées + regroupements demandés
EVENT_GROUPS = {
    "100m": ["100m"],
    "200m": ["200m", "200m Piste Courte"],
    "400m": ["400m", "400m Piste Courte"],
    "800m": ["800m", "800m Piste Courte"],
    "1500m": ["1 500m", "1 500m Piste Courte", "1500m", "1500m Piste Courte"],
    "3000m": ["3 000m", "3 000m Piste Courte", "3000m", "3000m Piste Courte"],
    "3000m Steeple": ["3000m Steeple (91)", "3 000m Steeple (91)", "3000m Steeple", "3 000m Steeple"],
    "5000m / 5km": ["5 000m", "5000m", "5 000m Piste Courte", "5000m Piste Courte", "5 Km Route", "5km Route", "5 km Route", "5km"],
    "10000m / 10km": ["10 000m", "10000m", "10 Km Route", "10km Route", "10 km Route", "10km"],
    "1/2 Marathon": ["1/2 Marathon", "Semi-Marathon", "Semi Marathon"],
    "Marathon": ["Marathon"],
}


def resolve_candidate_seqs(engine, athlete_identifier: str, limit: int = 10) -> list[str]:
    candidates = resolve_athlete_candidates(engine, athlete_identifier, limit=limit)
    seqs = candidates["seq"].dropna().astype(str).tolist() if not candidates.empty else []
    if athlete_identifier not in seqs:
        seqs.append(athlete_identifier)
    return list(dict.fromkeys(seqs))


def fetch_athlete_event_by_seq(engine, seq: str, event_label: str) -> pd.DataFrame:
    aliases = EVENT_GROUPS[event_label]
    query = """
        SELECT
            r.seq,
            a.name,
            a.birth_date_raw,
            a.birth_year,
            r.date,
            r.epreuve,
            r.perf,
            r.ville,
            r.tour
        FROM results r
        LEFT JOIN athletes a ON a.seq = r.seq
        WHERE r.seq = %(seq)s
          AND r.epreuve = ANY(%(aliases)s)
    """
    return pd.read_sql_query(query, engine, params={"seq": seq, "aliases": aliases})


def fetch_best_match_athlete_event(engine, athlete_identifier: str, event_label: str) -> pd.DataFrame:
    candidates = resolve_candidate_seqs(engine, athlete_identifier)

    best_df = pd.DataFrame()
    best_count = -1

    for seq in candidates:
        candidate_df = fetch_athlete_event_by_seq(engine, seq, event_label)
        count = len(candidate_df)
        if count > best_count:
            best_count = count
            best_df = candidate_df

    return best_df


def get_event_options_for_athletes(engine, athlete_1: str, athlete_2: str) -> list[str]:
    options = []
    for event_label in EVENT_GROUPS.keys():
        df1 = fetch_best_match_athlete_event(engine, athlete_1, event_label)
        df2 = fetch_best_match_athlete_event(engine, athlete_2, event_label)
        if (not df1.empty) or (not df2.empty):
            options.append(event_label)
    return options


def plot_compare_2_athletes_event(athlete_1: str, athlete_2: str, event_label: str, x_mode: str = "age"):
    engine = get_engine_from_config()

    a1_raw = fetch_best_match_athlete_event(engine, athlete_1, event_label)
    a2_raw = fetch_best_match_athlete_event(engine, athlete_2, event_label)

    a1 = prepare_for_comparison(a1_raw)
    a2 = prepare_for_comparison(a2_raw)

    if a1.empty or a2.empty:
        missing = []
        if a1.empty:
            missing.append(athlete_1)
        if a2.empty:
            missing.append(athlete_2)
        raise ValueError(f"Aucune donnée disponible sur '{event_label}' pour: {', '.join(missing)}")

    name_1 = a1["name"].dropna().iloc[0] if a1["name"].notna().any() else athlete_1
    name_2 = a2["name"].dropna().iloc[0] if a2["name"].notna().any() else athlete_2

    x_col = "age" if x_mode == "age" else "year"
    x_title = "Âge" if x_mode == "age" else "Année"

    a1_plot = a1.dropna(subset=[x_col]).sort_values(x_col)
    a2_plot = a2.dropna(subset=[x_col]).sort_values(x_col)

    fig = go.Figure()
    add_trace(fig, a1_plot, name_1, x_col=x_col, visible=True)
    add_trace(fig, a2_plot, name_2, x_col=x_col, visible=True)

    fig.update_layout(
        title=f"Comparaison de l'évolution des performances ({event_label})",
        xaxis_title=x_title,
        yaxis_title="Temps (secondes)",
        template="plotly_white",
        hovermode="x unified",
    )

    fig.update_yaxes(autorange=True)
    return fig


# -------------------- Interface interactive (Distance + Axe X) --------------------
ATHLETE_1 = "VAUDOIS"   # seq ou nom
ATHLETE_2 = "DIRAT"     # seq ou nom

engine = get_engine_from_config()
available_events = get_event_options_for_athletes(engine, ATHLETE_1, ATHLETE_2)

if not available_events:
    raise ValueError("Aucune course disponible entre 100m et Marathon pour au moins un des deux athlètes.")

default_event = "800m" if "800m" in available_events else available_events[0]

distance_dropdown = widgets.Dropdown(
    options=available_events,
    value=default_event,
    description="Distance:",
    layout=widgets.Layout(width="420px")
)

xmode_buttons = widgets.ToggleButtons(
    options=[("Âge", "age"), ("Année", "year")],
    value="age",
    description="Axe X:"
)

output = widgets.Output()


def refresh_chart(*_):
    with output:
        clear_output(wait=True)
        try:
            fig = plot_compare_2_athletes_event(
                ATHLETE_1,
                ATHLETE_2,
                distance_dropdown.value,
                xmode_buttons.value,
            )
            fig.show()
        except Exception as e:
            print(f"Erreur: {e}")


distance_dropdown.observe(refresh_chart, names="value")
xmode_buttons.observe(refresh_chart, names="value")

display(widgets.HBox([distance_dropdown, xmode_buttons]))
display(output)
refresh_chart()

Output()

In [4]:
# Cellule 3 : nuage de points avec option Toutes perfs / Meilleure par âge / Meilleure par année
import ipywidgets as widgets
from IPython.display import display, clear_output


# Reprend les mêmes athlètes par défaut
ATHLETE_1 = "VAUDOIS"   # seq ou nom
ATHLETE_2 = "DIRAT"     # seq ou nom


# Vérifie que les fonctions de la cellule 2 existent
required_funcs = [
    "get_engine_from_config",
    "fetch_best_match_athlete_event",
    "get_event_options_for_athletes",
    "prepare_for_comparison",
]
missing = [fname for fname in required_funcs if fname not in globals()]
if missing:
    raise ValueError(
        "Merci d'exécuter d'abord les cellules 1 et 2. Fonctions manquantes: " + ", ".join(missing)
    )


def apply_perf_mode(df: pd.DataFrame, mode: str) -> pd.DataFrame:
    """
    mode:
    - all: toutes les performances
    - best_age: meilleure perf par âge
    - best_year: meilleure perf par année
    """
    if df.empty:
        return df

    out = df.copy()

    if mode == "all":
        return out.sort_values("date")

    if mode == "best_age":
        out = out.dropna(subset=["age"]).copy()
        if out.empty:
            return out
        idx = out.groupby("age")["time_sec"].idxmin()
        return out.loc[idx].sort_values("age")

    if mode == "best_year":
        out = out.dropna(subset=["year"]).copy()
        if out.empty:
            return out
        idx = out.groupby("year")["time_sec"].idxmin()
        return out.loc[idx].sort_values("year")

    return out


def format_hovertemplate(athlete_label: str) -> str:
    return (
        "Athlète: " + athlete_label + "<br>"
        + "Perf: %{text}<br>"
        + "Date: %{customdata[0]|%Y-%m-%d}<br>"
        + "Âge: %{customdata[1]:.0f}<br>"
        + "Année: %{customdata[2]:.0f}<extra></extra>"
    )


def build_scatter_compare_event(athlete_1: str, athlete_2: str, event_label: str, x_mode: str = "date", perf_mode: str = "all"):
    engine = get_engine_from_config()

    a1_raw = fetch_best_match_athlete_event(engine, athlete_1, event_label)
    a2_raw = fetch_best_match_athlete_event(engine, athlete_2, event_label)

    # prepare_for_comparison conserve la règle métier birthdate incomplète -> meilleure par âge.
    # Ici on veut pouvoir afficher toutes les perfs : on repart des RAW pour garder ce contrôle.
    def prepare_raw(df_raw: pd.DataFrame) -> pd.DataFrame:
        if df_raw.empty:
            return df_raw
        out = df_raw.copy()
        out["date"] = pd.to_datetime(out["date"], errors="coerce")
        out["time_sec"] = out["perf"].apply(convert_time_to_seconds)
        out = out.dropna(subset=["date", "time_sec"]).sort_values("date")
        out["year"] = out["date"].dt.year
        out["birth_year"] = pd.to_numeric(out["birth_year"], errors="coerce")
        out["age"] = out["year"] - out["birth_year"]
        return out

    a1 = prepare_raw(a1_raw)
    a2 = prepare_raw(a2_raw)

    if a1.empty or a2.empty:
        missing_local = []
        if a1.empty:
            missing_local.append(athlete_1)
        if a2.empty:
            missing_local.append(athlete_2)
        raise ValueError(f"Aucune donnée disponible sur '{event_label}' pour: {', '.join(missing_local)}")

    a1 = apply_perf_mode(a1, perf_mode)
    a2 = apply_perf_mode(a2, perf_mode)

    name_1 = a1["name"].dropna().iloc[0] if a1["name"].notna().any() else athlete_1
    name_2 = a2["name"].dropna().iloc[0] if a2["name"].notna().any() else athlete_2

    x_map = {
        "date": "date",
        "age": "age",
        "year": "year",
    }
    x_title_map = {
        "date": "Date de compétition",
        "age": "Âge",
        "year": "Année",
    }

    x_col = x_map[x_mode]
    x_title = x_title_map[x_mode]

    a1_plot = a1.dropna(subset=[x_col]).sort_values(x_col)
    a2_plot = a2.dropna(subset=[x_col]).sort_values(x_col)

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=a1_plot[x_col],
            y=a1_plot["time_sec"],
            mode="markers",
            marker={"size": 9, "opacity": 0.75},
            name=name_1,
            text=a1_plot["perf"],
            customdata=a1_plot[["date", "age", "year"]],
            hovertemplate=format_hovertemplate(name_1),
        )
    )

    fig.add_trace(
        go.Scatter(
            x=a2_plot[x_col],
            y=a2_plot["time_sec"],
            mode="markers",
            marker={"size": 9, "opacity": 0.75},
            name=name_2,
            text=a2_plot["perf"],
            customdata=a2_plot[["date", "age", "year"]],
            hovertemplate=format_hovertemplate(name_2),
        )
    )

    mode_label = {
        "all": "Toutes les performances",
        "best_age": "Meilleure par âge",
        "best_year": "Meilleure par année",
    }[perf_mode]

    fig.update_layout(
        title=f"Nuage de points - {event_label} ({mode_label})",
        xaxis_title=x_title,
        yaxis_title="Temps (secondes)",
        template="plotly_white",
        hovermode="closest",
    )

    fig.update_yaxes(autorange=True)
    return fig


# -------------------- Interface interactive --------------------
engine = get_engine_from_config()
available_events = get_event_options_for_athletes(engine, ATHLETE_1, ATHLETE_2)

if not available_events:
    raise ValueError("Aucune course disponible entre 100m et Marathon pour au moins un des deux athlètes.")

default_event = "800m" if "800m" in available_events else available_events[0]

distance_dropdown_3 = widgets.Dropdown(
    options=available_events,
    value=default_event,
    description="Distance:",
    layout=widgets.Layout(width="420px")
)

xmode_buttons_3 = widgets.ToggleButtons(
    options=[("Date", "date"), ("Âge", "age"), ("Année", "year")],
    value="date",
    description="Axe X:"
)

perf_mode_buttons_3 = widgets.ToggleButtons(
    options=[
        ("Toutes", "all"),
        ("Best Âge", "best_age"),
        ("Best Année", "best_year"),
    ],
    value="all",
    description="Filtre:"
)

output_3 = widgets.Output()


def refresh_chart_3(*_):
    with output_3:
        clear_output(wait=True)
        try:
            fig = build_scatter_compare_event(
                ATHLETE_1,
                ATHLETE_2,
                distance_dropdown_3.value,
                xmode_buttons_3.value,
                perf_mode_buttons_3.value,
            )
            fig.show()
        except Exception as e:
            print(f"Erreur: {e}")


distance_dropdown_3.observe(refresh_chart_3, names="value")
xmode_buttons_3.observe(refresh_chart_3, names="value")
perf_mode_buttons_3.observe(refresh_chart_3, names="value")

controls_row_1 = widgets.HBox([distance_dropdown_3, xmode_buttons_3])
controls_row_2 = widgets.HBox([perf_mode_buttons_3])

display(controls_row_1)
display(controls_row_2)
display(output_3)
refresh_chart_3()

Output()